# Internet Of Water SPARQL

A simple document to explore IoW queries




In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import numpy as np
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
import folium

# set SPARQL endpoint(s)
iow = "https://graph.geoconnex.us/repositories/iow"


In [2]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [3]:
#  <https://geoconnex.us/ref/gages/1012193>
#  <https://geoconnex.us/ref/gages/1118104>

cq = """
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

select DISTINCT ?mainstem ?gage ?gsourl ?gname ?name ?gwkt ?mswkt where {
    <https://geoconnex.us/ref/gages/1118104> hyf:referencedPosition ?rp .
    ?rp hyf:HY_IndirectPosition ?ip .
    ?ip hyf:linearElement ?mainstem .
    BIND (?mainstem as ?target)
    ?gage hyf:referencedPosition ?rp2 .
    ?rp2 hyf:HY_IndirectPosition ?ip2.
    ?ip2 hyf:linearElement ?target .
    ?gage schema:name ?gname .
    ?gage schema:subjectOf ?gso .
    ?gso schema:url ?gsourl .
    ?mainstem schema:name ?name .
    ?mainstem geo:hasGeometry ?msgeom .
    ?msgeom geo:asWKT ?mswkt .
    ?gage geo:hasGeometry ?ggeom .
    ?ggeom geo:asWKT ?gwkt
        }
"""

dfsc = get_sparql_dataframe(iow, cq)
dfsc.head()

,mainstem,gage,gsourl,gname,name,gwkt,mswkt
0,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1118105,https://waterdata.usgs.gov/monitoring-location...,"Mill Race at Amana, IA",Iowa River,POINT (-91.86517779999998 41.79611839999999),LINESTRING (-93.69864583166763 43.143736799699...
1,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1067246,https://waterdata.usgs.gov/monitoring-location...,"Iowa River near Iowa Falls, IA",Iowa River,POINT (-93.2738146 42.522202),LINESTRING (-93.69864583166763 43.143736799699...
2,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1030542,https://waterdata.usgs.gov/monitoring-location...,"East Branch Iowa River above Hayfield, IA",Iowa River,POINT (-93.6893825 43.15579259999999),LINESTRING (-93.69864583166763 43.143736799699...
3,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1118115,https://waterdata.usgs.gov/monitoring-location...,"Iowa Riv blw Iowa Riv Power Dam at Coralville, IA",Iowa River,POINT (-91.5649749 41.6736269),LINESTRING (-93.69864583166763 43.143736799699...
4,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1017821,https://waterdata.usgs.gov/monitoring-location...,"Iowa River at Columbus Junction, IA",Iowa River,POINT (-91.34682159999998 41.27835889),LINESTRING (-93.69864583166763 43.143736799699...


In [4]:
# gdf = geopandas.GeoDataFrame(df_s3urmi, geometry=geopandas.points_from_xy(df_s3urmi.lng, df_s3urmi.lat), crs=3857)
dfsc['geometry'] = gpd.GeoSeries.from_wkt(dfsc['gwkt'])
geo_df = gpd.GeoDataFrame(dfsc, geometry='geometry')

geo_df['lon'] = geo_df['geometry'].x
geo_df['lat'] = geo_df['geometry'].y

In [5]:
geo_df.head()


,mainstem,gage,gsourl,gname,name,gwkt,mswkt,geometry,lon,lat
0,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1118105,https://waterdata.usgs.gov/monitoring-location...,"Mill Race at Amana, IA",Iowa River,POINT (-91.86517779999998 41.79611839999999),LINESTRING (-93.69864583166763 43.143736799699...,POINT (-91.86518 41.79612),-91.865178,41.796118
1,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1067246,https://waterdata.usgs.gov/monitoring-location...,"Iowa River near Iowa Falls, IA",Iowa River,POINT (-93.2738146 42.522202),LINESTRING (-93.69864583166763 43.143736799699...,POINT (-93.27381 42.52220),-93.273815,42.522202
2,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1030542,https://waterdata.usgs.gov/monitoring-location...,"East Branch Iowa River above Hayfield, IA",Iowa River,POINT (-93.6893825 43.15579259999999),LINESTRING (-93.69864583166763 43.143736799699...,POINT (-93.68938 43.15579),-93.689382,43.155793
3,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1118115,https://waterdata.usgs.gov/monitoring-location...,"Iowa Riv blw Iowa Riv Power Dam at Coralville, IA",Iowa River,POINT (-91.5649749 41.6736269),LINESTRING (-93.69864583166763 43.143736799699...,POINT (-91.56497 41.67363),-91.564975,41.673627
4,https://geoconnex.us/ref/mainstems/324976,https://geoconnex.us/ref/gages/1017821,https://waterdata.usgs.gov/monitoring-location...,"Iowa River at Columbus Junction, IA",Iowa River,POINT (-91.34682159999998 41.27835889),LINESTRING (-93.69864583166763 43.143736799699...,POINT (-91.34682 41.27836),-91.346822,41.278359


In [6]:
from folium.plugins import MarkerCluster

map = folium.Map(location=[geo_df.centroid[0].y, geo_df.centroid[0].x], tiles='CartoDB positron', zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)

locations = geo_df[['lat', 'lon']]
locationlist = locations.values.tolist()

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup='Gage:'+geo_df['gage'][point]+' '+geo_df['gname'][point], icon=folium.Icon(color="red", icon_color='white', icon='gauge-simple', angle=0, prefix='fa')).add_to(marker_cluster)

map